In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# from jupyter_dash import JupyterDash
import plotly.express as px
import nest_asyncio
from dash import Dash
# Habilita que funcione correctamente en Jupyter
nest_asyncio.apply()


C:\Users\Alejandro\AppData\Local\Temp\ipykernel_12924\960488587.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_12924\960488587.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
ventas1 = pd.read_csv(r'Ventas.csv', dtype = str, encoding= 'utf-8', sep = ',')
vendedor1 = pd.read_csv(r'Vendedor.csv', dtype = str, encoding= 'utf-8', sep = ';')
proveedor1 = pd.read_csv(r'Proveedores.csv', dtype = str, encoding= 'utf-8', sep = ';')
producto1 = pd.read_csv(r'Estructura_Comercial.csv', dtype = str, encoding= 'utf-8', sep = ';')
cliente1 = pd.read_csv(r'Clientes.csv', dtype = str, encoding= 'utf-8', sep = ';')

In [3]:
ventas = ventas1.copy()
vendedor = vendedor1.copy()
proveedor = proveedor1.copy()
producto = producto1.copy()
cliente = cliente1.copy()

In [4]:
cliente = cliente[['nit','nombre']]
producto = producto[['CODIGO', 'NOMBRE']]

In [5]:
ventas = ventas.merge(vendedor, how = 'left', left_on = 'Vendedor', right_on = 'codigo')
ventas = ventas.merge(cliente, how = 'left', left_on = 'Nit', right_on = 'nit')
ventas = ventas.merge(producto, how = 'left', left_on = 'Producto', right_on = 'CODIGO')

In [6]:
ventas = ventas[['Fecha', 'nombre', 'NOMBRE', 'Nombre_Vendedor', 'TCantidad', 'TNeto', 'Mes', 'Anio']]
ventas["TCantidad"] = ventas["TCantidad"].str.replace(",", ".").str.strip()
ventas["TNeto"] = ventas["TNeto"].str.replace(",", ".").str.strip()
ventas["TCantidad"] = pd.to_numeric(ventas["TCantidad"], errors="coerce")
ventas["TNeto"] = pd.to_numeric(ventas["TNeto"], errors="coerce")
ventas.columns = ['Fecha', 'Cliente', 'Producto', 'Vendedor', 'Cantidad', 'Valor', 'Mes', 'Anio']
ventas2024 = ventas[ventas['Anio'] == '2024']

In [7]:
ventas_producto24 = ventas2024.groupby("Producto").agg(
    Unidades=("Cantidad", "sum"),
    Pesos=("Valor", "sum")
).reset_index()

ventas_cliente24 = ventas2024.groupby("Cliente").agg(
    Unidades=("Cantidad", "sum"),
    Pesos=("Valor", "sum")
).reset_index()

ventas_x_mes = ventas2024.groupby("Mes").agg(
    Pesos=("Valor", "sum")
).reset_index()
ventas_x_mes['Mes'] = ventas_x_mes['Mes'].astype(int)
ventas_x_mes = ventas_x_mes.sort_values('Mes').reset_index(drop=True)
ventas_x_mes['Mes'] = ventas_x_mes['Mes'].map({1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio',
                                                7: 'Julio', 8: 'Agosto', 9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'})

ventas_vendedor24 = ventas2024.groupby("Vendedor").agg(
    Pesos=("Valor", "sum")
).reset_index()
ventas_vendedor24 = ventas_vendedor24.sort_values('Pesos', ascending=False).reset_index(drop=True)
ventas_vendedor24 = ventas_vendedor24.head(10)

In [8]:
fig_vendedores = px.pie(
    ventas_vendedor24,
    names="Vendedor",
    values="Pesos",
    title=" ",
    hole=0.3  
)

# Mostrar el porcentaje directamente
fig_vendedores.update_traces(
    textposition='inside',
    textinfo='percent+label'
)

# Estilo opcional
fig_vendedores.update_layout(
    height=500,
    margin=dict(l=40, r=40, t=50, b=40),
    font_color="black",
    font_family="Lato"
)

fig_vendedores.show()

In [9]:
# ----- APP DASH -----
app = Dash(__name__)
server = app.server

app.title = "Dashboard de Ventas"

# ----- LAYOUT COMPLETO -----
app.layout = html.Div([
    html.H1("Análisis de Ventas 2024", style={"textAlign": "center"}),

    html.Div([
        html.Label("Agrupar por:"),
        dcc.RadioItems(
            id="grupo-selector",
            options=[
                {"label": "Producto", "value": "producto"},
                {"label": "Cliente", "value": "Cliente"}
            ],
            value="producto",
            inline=True,
        ),
    ], style={"margin": "20px"}),

    html.Div([
        html.Label("Métrica:"),
        dcc.RadioItems(
            id="metrica-selector",
            options=[
                {"label": "Unidades", "value": "Unidades"},
                {"label": "Pesos", "value": "Pesos"}
            ],
            value="Unidades",
            inline=True
        ),
    ], style={"margin": "20px"}),

    html.Div([
        html.Label("Ver desde el número:"),
        dcc.Slider(
            id='inicio-slider',
            min=0,
            max=90,  # Esto puede cambiarse dinámicamente si quieres
            step=1,
            value=0,
            marks={0: "0", 25: "25", 50: "50", 75: "75"},
            tooltip={"placement": "bottom", "always_visible": True}
        )
    ], style={"margin": "30px"}),

    dcc.Graph(id="grafico-ventas"),
    
    html.H2("Ventas Mensuales Totales (Pesos)", style={"textAlign": "center", "marginTop": "60px"}),

    dcc.Graph(
        id="grafico-lineas-mensual",
        figure=px.line(ventas_x_mes, x="Mes", y="Pesos", markers=True)
        .update_layout(
            xaxis_title="Mes",
            yaxis_title="Ventas en Pesos",
            height=400,
            margin=dict(l=40, r=40, t=40, b=40)
        )
    ),

    html.H2("Participación de Vendedores (2024)", style={"textAlign": "center", "marginTop": "60px"}),

    dcc.Graph(
        id="grafico-vendedores-pie",
        figure=fig_vendedores
    )

], style={'height': '1000px', 
        "color": "black",             
        "fontFamily": "'Nunito', sans-serif"})

In [10]:
@app.callback(
    Output("grafico-ventas", "figure"),
    Input("grupo-selector", "value"),
    Input("metrica-selector", "value"),
    Input("inicio-slider", "value")
)
def actualizar_grafico(grupo, metrica, inicio):
    if grupo == "producto":
        df = ventas_producto24.sort_values(by=metrica, ascending=False)
        eje_x = "Producto"
    else:
        df = ventas_cliente24.sort_values(by=metrica, ascending=False)
        eje_x = "Cliente"

    # Ventana de 10 elementos
    fin = inicio + 10
    df_vista = df.iloc[inicio:fin]

    fig = px.bar(df_vista, x=eje_x, y=metrica, title=f"Ventas por {grupo.capitalize()} ({metrica})")
    fig.update_layout(
        xaxis_title=eje_x,
        yaxis_title=metrica,
        xaxis_tickangle=-45,
        height=500,
        margin=dict(l=20, r=20, t=50, b=250),
    )
    fig.update_xaxes(tickfont=dict(size=10), automargin=True)

    return fig


In [11]:
app.run(mode='external', debug=True, port=8050)